# Intro to Crypto by Breaking Things - Implementing RSA for real

## How to set-up RSA for Dummies
The first problem we have is generating primes of sufficient size. Let's start by making a little function to check if big numbers are prime

In [170]:
from random import randint
smallPrimes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 
            127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 
            257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 
            401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]

def isPrime(n):
    # Quickly test small factors.
    if n in smallPrimes:
            return True

    # Use Fermat's little theorem to do the harder work on larger primes. (https://en.wikipedia.org/wiki/Fermat%27s_little_theorem)
    for i in range(0, 50): 
        base = randint(2, n-1)
        if pow(base, n-1, n) != 1: 
            return False
    
    return True

And let's check we've not been completely stupid by checking a few cases:

In [171]:
print(2, isPrime(2))
print(3, isPrime(3))
print(10, isPrime(10))

print(65537, isPrime(65537))
print(65539, isPrime(65539))
print(65541, isPrime(65541))



2 True
3 True
10 False
65537 True
65539 True
65541 False


And now for something a bit bigger. 

This prime was nicked from the [RFC 3526](https://www.rfc-editor.org/rfc/rfc3526.txt). 

In [172]:
bigPrime = """FFFFFFFF FFFFFFFF C90FDAA2 2168C234 C4C6628B 80DC1CD1
      29024E08 8A67CC74 020BBEA6 3B139B22 514A0879 8E3404DD
      EF9519B3 CD3A431B 302B0A6D F25F1437 4FE1356D 6D51C245
      E485B576 625E7EC6 F44C42E9 A637ED6B 0BFF5CB6 F406B7ED
      EE386BFB 5A899FA5 AE9F2411 7C4B1FE6 49286651 ECE45B3D
      C2007CB8 A163BF05 98DA4836 1C55D39A 69163FA8 FD24CF5F
      83655D23 DCA3AD96 1C62F356 208552BB 9ED52907 7096966D
      670C354E 4ABC9804 F1746C08 CA18217C 32905E46 2E36CE3B
      E39E772C 180E8603 9B2783A2 EC07A28F B5C55DF0 6F4C52C9
      DE2BCBF6 95581718 3995497C EA956AE5 15D22618 98FA0510
      15728E5A 8AACAA68 FFFFFFFF FFFFFFFF"""

bigPrime = bigPrime.replace(" ", "")
bigPrime = bigPrime.replace("\n", "")

bigDaddy = int(bigPrime, 16)

isPrime(bigDaddy)

True

Alright, so how do we generate a large random prime. The code below implements the process:

In [173]:
def bigPrime(bits):
    from random import getrandbits
    p = getrandbits(bits)
    while not isPrime(p):
        p = getrandbits(bits)
    return p

Does this prove that the numbers are prime? Of course not, there sort of "industrial strength" primes.

Alrighty then, let's generate some primes:

In [174]:
p = bigPrime(1024)
q = bigPrime(1024)

p,q

(15859894126437098818729417319725393142615345147740989046728938476790721354365190626554389244876419737824122176360076873927375794725919786734523218209382928616033349517006134115080690193656658489576809582426047751294110196688109198253694398143718035497495523329724414237667677725223080746722746676311890734649,
 33823204988968627431881166892074624010419927676376120685519679105356111809215708466251194448376567771621880468108056055202167852949961258160124313616523776899184032730383123954489000713905781847277909018098816199831396251695092123974361921388240062391888035945815916791830746996716741613327777392944404069529)

And multiply to get a modulus:

In [175]:
n = p * q
n

536432450141821511951495705538773680618247714414910398561564547312025022973744078834226241149732348684828324609798683468793872957339160114565359628321870807456303540519149960356885212537107345005606172257161249951649253355159073040526180734325391148213060136030150874724916607091051188557267229372552268928769490373016958710782780666358046799146949644142359519982855495564524991033330919744601166986743489484364619230756793526589354119190491749523853036125980506598407148895470420544771708611128212167351382563592394018583412075485273464428739245948293072429679750702055165392800679837789063410721770062905085410321

Now we need a few house keeping functions:

  * Extended GCD
  * LCM

The LCM is just the algorithm you saw in MU123 etc.


# Explanation of the role of Extended GCD

It's the standard GCD algorithm extended to show more of its internal state. It's used to find solutions to the equations of the form:

$$(ax + by) = gcd(x, y)$$

In our case, we're looking for a solution such that:

$$e \times d \equiv {1 \mod {\text{lcm} (p-1,q-1)}}$$

So, if we substitute:

$$ex + (b \times {\text{lcm} (p-1,q-1)}) = gcd(e, {\text{lcm} (p-1,q-1)})$$

But by requirement for the scheme to work:

$$gcd(e, {\text{lcm} (p-1,q-1)}) = 1$$

So we can rewrite the right hand side to 1:

$$(e \times x) + (b \times {\text{lcm} (p-1,q-1)}) = 1$$

Now we take both sides  $\mod {\text{lcm} (p-1,q-1)}$:

$$((e \times x) + (b \times {\text{lcm} (p-1,q-1)})) \mod {\text{lcm} (p-1,q-1)} \equiv 1 \mod {\text{lcm} (p-1,q-1)}$$

But notice that we just have $b$ multiplied by the thing in the mod, so we can simplify:

$$ex \equiv 1 \mod {\text{lcm} (p-1,q-1)}$$

This looks like our target equation above, so $x=d$ which is our encryption exponent.

In [222]:
def gcdExtended(a, b):
		if a == 0 :   
			return b,0,1
		     
		gcd,x1,y1 = gcdExtended(b % a, a)  
     
		x = y1 - (b//a) * x1  
		y = x1  
	     
		return gcd,x,y 

def lcm(a, b):
	gcd, _, _ = gcdExtended(a, b)
	return (a * b) // gcd


Right, now we get to the business end of setting up RSA. First we need to compute phi(n), which is the number of numbers relatively prime to "n"

In [177]:
phi_n = lcm(p-1, q-1)

Next we need to define what we want for the encryption exponent. It needs to be big enough so that when we do m^e it is bigger than p and thus "loops around"

In [178]:
e = 65537

Now the final step, let's compute d from phi_n and e.

In [186]:
_, d, _ = gcdExtended(e, phi_n)

if d < 0:
	d = phi_n + d # d is negative. Lost a good 45 minutes there.

d

22293373697568070181559920212979575901763506586711481060809027574846486003278510104442825366608782339544311543332208359593721290773652592688604568681632594991503650312746537235104162335724537170466974547414602581741011911888629786380260295749378114134790971710562272856137006484708855965855729703828641430962425577709997669217334204189779382510767582122795489336082047668412571680546834612474582992015702098683762105373002322653714333818743329423008679702294711483211052202372401180196257007001759697603437392410543686466881886775651635538723698604768151502697929633245975033076721616642581887432053227481448453769

Alright, let's check everything works!!!

In [194]:
m = 420

cipherText = pow(m, e, n)
plainText = pow(cipherText, d, n)

m, e, plainText

(420, 65537, 420)

# Now let's put the whole thing in to one neat package

In [218]:
class RSAFixedKey:
    def __init__(self):
        self.p = bigPrime(1024)
        self.q = bigPrime(1024)
        self.n = self.p * self.q
        self.e = 65537
        phi_n = lcm((self.p-1),(self.q-1))
        _, self.d, _ = gcdExtended(e, phi_n)
        if self.d < 0:
            self.d = phi_n + self.d

    def encrypt(self, data):
        return pow(data, self.e, self.n)

    def decrypt(self, data):
        return pow(data, self.d, self.n)

And let's test it out!

In [220]:
rsa = RSAFixedKey()

plainTextBefore = 69
cipherText=rsa.encrypt(69)
plainTextAfter=rsa.decrypt(cipherText)

plainTextBefore, cipherText, plainTextAfter


(69,
 3413799958950923099492517674653162373327160788008035694051054676016163442439556708615166295496897323600675639732596321403010717076917115271630388171217236480545534853121114200905068642757632608462703523143735805044607322914972460974507253602478773888552008630473763026420562133034489871516607626598256253703591595489610786656559080292193258497500839090398658732957126079502817295868058326486094470469847704997233531286471513836682076661214378110722599300479179186092524614732750474620229874733250025693390923956687023213123574045376240110682528517290120062635586511292814670173475779999285528988717516409971292798594,
 69)